In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import shutil
sns.set()

In [ ]:
def quasar_z_range(redshift, sdss_id, noise):
    
    cut_redshifts = (redshift >= 1.8) & (redshift <= 2.2) # gives boolean list with T in the range 1.8-2.2
    
    filtered_ids = np.array(sdss_id)[np.array(cut_redshifts)] # gives the values in the ID array for the Ts in cut list
    filtered_redshifts = np.array(redshift)[np.array(cut_redshifts)] # same as above but for the redshift array
    filtered_noise = np.array(noise)[np.array(cut_redshifts)] # same as above but for the noise array
    
    cut_values = {"ID":filtered_ids, "REDSHIFTS":filtered_redshifts, "NOISE":filtered_noise} # puts these new lists
                                                                                             # into a dictionary
    
    return cut_values

cut_lists = quasar_z_range(redshift, sdss_id, noise)

print("The SDSS ID's that are in the z range is "  +str(cut_lists["ID"]))
print("The redshifts values that are in the z range is "  +str(cut_lists["REDSHIFTS"]))

In [ ]:
def writeresultsdict(resultsdict, outfileroot, outfileprefix=None):

    mainoutfilename = outfileroot + '_donut_summary.txt'
    with open(mainoutfilename, 'w') as fout:
        donutsum = resultsdict['donut_summary']
        for key in donutsum.keys():
            outstr = key + ' ' + "{:f}".format(donutsum[key])+'\n'
            fout.write(outstr)
    # now loop through and write out the results for each zernike
    # write the numbers in one file, and the arrays one per file 
    for zdictname in ['z4ResultDict','z5ResultDict','z6ResultDict',\
                      'z7ResultDict','z8ResultDict']:
        zdict = resultsdict[zdictname]
        for zdictkey in ['thetax','thetaxErr','thetay','thetayErr',\
                         'delta','deltaErr','meanDeltaBefore',\
                         'rmsDeltaBefore','meanDeltaAfter',\
                         'rmsDeltaAfter']:
            zoutfilename = outfileroot + '_' + zdictname + '.txt'
            with open(zoutfilename, 'w') as fout:
                outstr = zdictkey + ' ' + "{:f}".format(zdict[zdictkey])+'\n'
                fout.write(outstr)
        for zarrayname in ['deltaArrayX','deltaArrayY',\
                           'deltaArrayBefore','deltaArrayAfter']:
            zarray = zdict[zarrayname]
            arroutname = outfileroot + '_' + zdictname + '_' + zarrayname + '.txt'
            hdrstr = outfileroot + ' ' + zdictname + ' ' + zarrayname
            np.savetxt(arroutname,zarray,header=hdrstr)
# from Professor Rockosi's email from July 14, 2019

In [ ]:
def quasar_z_range(redshifts, psfmag, plate, mjd, fiberid):
    
    fits_image_totalQ = fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/DR14_Q_spectra/DR14Q_v4_4.fits")
    hdul = fits_image_totalQ # this block of code is just reading in the data
    quasar_data = hdul["DR14Q_v4_4"].data
    
    redshift = quasar_data.field(str(redshifts)) # making the redshift vector
    psfmagtable = quasar_data.field(str(str(psfmag))) # the light magnitudes are in a table
    rmagvec = psfmagtable[:,2] # the r band is the 3 entry in the table, starts at 0 index
    
    plate = quasar_data.field(str(plate)) # making the plate vector
    mjd = quasar_data.field(str(mjd)) # making the mjd vector
    fiber = quasar_data.field(str(fiberid)) # making fiber vector
    
    cut_redshifts = (redshift >= 1.8) & (redshift <= 2.2) # gives boolean list with T in the range 1.8-2.2
    cut_rband = rmagvec > 17. # the range we're looking for in r band
    # ^ this range is opposite than what Professor Rockosi said, but it's the only way to get a sizable sample size
    cutlist = (cut_rband) & (cut_redshifts) # ma
    
    filtered_plate = np.array(plate)[np.array(cutlist)] # gives the values in the ID array for the Ts in cut list
    filtered_mjd = np.array(mjd)[np.array(cutlist)] # same as above but for the redshift array
    filtered_fiberid = np.array(fiberid)[np.array(cutlist)] # same as above but for the noise array
    
    cut_values = {"PLATE":filtered_plate, "MJD":filtered_mjd, "NOISE":filtered_} # puts these new lists
                                                                                             # into a dictionary
    
    return cut_values



In [ ]:
"""
A quick example for using get_filenames. The idea is , it takes in a
string as a path (read: folder), and returns all the names in the
directory indicated by the path.

Note that this assumes a Linux operating system, so smooth operation
is not guaranteed in Mac.
"""

import os
import re


def get_filenames(path='.', extension=None, pattern=None, identifiers=None, include_path=False):
    """
    Retrieves a list containing the filenames from a target directory.

    By default this retrieves all entries in the directory, hereafter referred
    to as filenames. Specific file types or folders can be retrieved by
    filtering by extension, a portion of the filename, or a list of
    identifiers. Also supports Regular Expression patterns.

    Parameters
    ----------
    path: string or path-like object, optional
        The path from which to retrieve the filepaths. Default behavior is to
        list the current directory.
    extension: string, optional
        The extension of the filenames to be retrieved. This works by comparing
         the end of the entry names to the string specified, so it need not be
        an extension, merely the end of entry-name you wish to retrieve.
    pattern: string, optional
        A pattern by which to filter what filenames and entries are returned. This
        can be the whole filename, or just a portion. Alternately, a Regular
        Expression can be provided. All entries that match in this fashion will be
        returned.
    identifiers: list or tuple, optional
        Instead of a single string, a list of strings or numbers can be
        provided. The list can contain whole filenames, or just portions of the
        filenames. If a number is passed, the number is converted to a string.
        This does not support Regular Expressions.
    include_path: bool, optional
        If True, the filenames are returned with the path appended to them.

    Returns
    -------
    filename_list: a list of strings that contain the filenames after filtering

    See Also
    --------
    os.listdir: returns the names of all entries in a directory
    re: module that supports regular expression matching operations for python

    Examples
    --------

    Retrieve a list of file names using a file extension, and a Regex pattern.

    >>> biasframe_path = '/home/lee/Documents/bias_frames/'
    >>> extension = '.fits.fz'
    >>> pattern = '(?=.*k4m)'  # look-ahead regex pattern that checks for 'k4m'
    >>> fits_list = get_filenames(biasframe_path, extension=extension, pattern=pattern)
    >>> print(fits_list)
    ['k4m_180211_231642_zri.fits.fz', 'k4m_180211_225927_zri.fits.fz', 'k4m_180211_231335_zri.fits.fz', 'k4m_180211_230119_zri.fits.fz', 'k4m_180211_231949_zri.fits.fz', 'k4m_180211_231834_zri.fits.fz', 'k4m_180211_231449_zri.fits.fz', 'k4m_180211_231604_zri.fits.fz', 'k4m_180211_231527_zri.fits.fz', 'k4m_180211_231911_zri.fits.fz', 'k4m_180211_230004_zri.fits.fz', 'k4m_180211_231719_zri.fits.fz', 'k4m_180211_231412_zri.fits.fz', 'k4m_180211_230042_zri.fits.fz', 'k4m_180211_231756_zri.fits.fz']

    Retrieve a list of file names using a sequence of numbers

    >>> identifiers = np.arange(190800, 190900)
    >>> fits_list = get_filenames(biasframe_path, identifiers=identifiers)
    >>> fits_list
    ['c4d_170331_190830_zri.fits.fz', 'c4d_170331_190853_zri.fits.fz']


    """
    # retrieve all filenames from the directory
    filename_list = os.listdir(path)

    # keep all filenames with the proper extension
    if extension is not None:
        filename_list = [filename for filename in filename_list if
                         filename[-len(extension):] == extension]

    # keep all filenames that match the pattern
    if pattern is not None:
        filename_list = [filename for filename in filename_list if re.search(pattern, filename)]

    # keep all filenames that match the identifiers provided
    if identifiers is not None:
        storage_list = []
        try:
            for ident in identifiers:
                storage_list.extend([filename for filename in filename_list if str(ident) in filename])

        except TypeError:
            print(identifiers, 'is not a list, tuple, or otherwise iterable')
        else:
            filename_list = storage_list

    if include_path:
        filename_list = [path + filename for filename in filename_list]

    return filename_list

'''example code'''

# outer path
master_dir = '/home/lee/Data/darkcurrent_25c/'
# retrieve the list of directories that contain exposures.
# conveniently, I have ended the name of everything that
# has what I want with 'exposure', so I can take advantage
# of how the extension variable looks at the end of the name
sub_dir_list = get_filenames(master_dir, extension='exposure')
print(sub_dir_list) # print to check the output
'''
>>> print(sub_dir_list) # print to check the output
['60s_exposure', '30s_exposure', '240s_exposure', '15s_exposure', '120s_exposure']
'''

# this makes a list of lists, with each entry in the outer list corresponding to a
# subdirectory, and each inner list being the file names withing that subdirectory
sub_dir_filenames = []
for sub_dir in sub_dir_list:
    path = master_dir + sub_dir + '/'  # adding a slash, to account for how os.listdir(path) does not include one
    # retrieve the filenames, checking the extension to make sure
    # this time, include path is turned on, so the entire directory pointing to the file will be included
    sub_dir_filenames.append(get_filenames(path, extension='.fits', include_path=True))

# print to check the output
for filename_list in sub_dir_filenames: print(filename_list)


In [ ]:
def datafrommultiplefolders(foldername,start,end):
    # create an empty listt
    sub_dir_list_fin = []

    # goes through all the folders with their names in the start and end of the range. Slight modification
    # to Lee's code.
    for i in range(start,end):
        try:
            master_dir = "/Users/matt/Desktop/DESI_Research/DESI_ML/"+str(foldername)+'/' + str(i)
            sub_dir_list_fin.append(get_filenames(path = master_dir, extension = '.fits')) 
            
        except:
            pass

    # this flatttens the list. This is b/c w/o doing this you have a list of lists
    flat_list = [item for sublist in sub_dir_list_fin for item in sublist]

    return flat_list

In [ ]:
# this code copies the fits in their respective folders into one folder

def copyfiles_fromfolder_tofolder(Root_dir,target_folder,extension)
    RootDir1 = str(Root_dir)
    TargetFolder = str(target_folder)
    for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):
        for name in files:
            if name.endswith(str(extension)):
                SourceFolder = os.path.join(root,name)
                shutil.copy2(SourceFolder, TargetFolder)


# this is the function code w/ example

RootDir1 = "/Users/matt/Desktop/DESI_Research/DESI_ML/Valid_Quasar_Data"
TargetFolder = "/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars"
for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):
        for name in files:
            if name.endswith('.fits'):
                #print("Found")
                SourceFolder = os.path.join(root,name)
                shutil.copy2(SourceFolder, TargetFolder) #copies csv to new folder



In [ ]:
def fitsfromfolders(foldername,start,end):
    sub_dir_list_fin = []
    for i in range(start,end):
        try:
            master_dir = "/Users/matt/Desktop/DESI_Research/DESI_ML/"+str(foldername)+'/' + str(i)
            sub_dir_list_fin.append(get_filenames(path = master_dir, extension = '.fits')) 
        except:
            pass
    flat_list = [item for sublist in sub_dir_list_fin for item in sublist]
    
    
    return flat_list

'''
vfits_listD15 = fitsfromfolders("Valid_Quasar_Data", 3585, 10001)
print(len(vfits_listD15))
nvfits_listD15 = fitsfromfolders("DR15_wrong_rband_Quasars", 3585, 10001)
print(len(nvfits_listD15))
nvfits_listD14 = fitsfromfolders("DR14_wrong_rband", 3585, 10001)
print(len(nvfits_listD15))

reading_in("DR15_wrong_rband_Quasars", nvfits_listD15, 3585, 10001)
'''

In [ ]:
quasardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/", extension='.fits')
batched_quasardata = np.array_split(np.array(quasardata),9)

%%time 

noise = []
for i in range(len(quasardata)):
    with fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/" +str(quasardata[i])+ "") as hdul:
        data = hdul['SPALL'].data
        SN = data.field("SN_MEDIAN_ALL")
        noise.extend(SN)  #np.extend just adds another item to the list
        hdul.close()
        del hdul['SPALL'].data
        del hdul['COADD'].data
        del hdul['PRIMARY'].data 
        del hdul
#     hdul.close()

print(len(noise))

'''
7974
CPU times: user 14min 16s, sys: 12 s, total: 14min 28s
Wall time: 15min 27s
'''

In [ ]:
quasardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/", extension='.fits')
batched_quasardata = np.array_split(np.array(quasardata),9)

%%time

noise = []
for i in range(len(batched_quasardata)):
    for j in range(len(batched_quasardata[0])):
        with fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/" +str(batched_quasardata[i][j])+ "") as hdul:
            data = hdul['SPALL'].data
            SN = data.field("SN_MEDIAN_ALL")
            noise.extend(SN)  #np.extend just adds another item to the list
            hdul.close()
            del hdul['SPALL'].data
            del hdul['COADD'].data
            del hdul['PRIMARY'].data   
            del hdul
#     hdul.close()

print(len(noise))

'''
7974
CPU times: user 14min 49s, sys: 11 s, total: 15min
Wall time: 15min 46s
'''

In [ ]:
quasardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/", extension='.fits')
batched_quasardata = np.array_split(np.array(quasardata),9)

%%time 

noise = []
for i in range(len(quasardata)):
    with fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/" +str(quasardata[i])+ "") as hdul:
        data = hdul['SPALL'].data
        SN = data.field("SN_MEDIAN_ALL")
        noise.extend(SN)  #np.extend just adds another item to the list
        hdul.close()
        #del hdul['SPALL'].data
        #del hdul['COADD'].data
        #del hdul['PRIMARY'].data 
        del hdul
#     hdul.close()

print(len(noise))

'''
7974
CPU times: user 15min 48s, sys: 25.6 s, total: 16min 13s
Wall time: 17min 34s
'''

In [2]:
quasardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/", extension='.fits')
batched_quasardata = np.array_split(np.array(quasardata),9)

%%time

noise = []
for i in range(len(batched_quasardata)):
    for j in range(len(batched_quasardata[0])):
        with fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/" +str(batched_quasardata[i][j])+ "") as hdul:
            data = hdul['SPALL'].data
            SN = data.field("SN_MEDIAN_ALL")
            noise.extend(SN)  #np.extend just adds another item to the list
            del hdul
            
#             del hdul['SPALL'].data
#             del hdul['COADD'].data
#             del hdul['PRIMARY'].data   
#     hdul.close()

print(len(noise))

'''
7974
CPU times: user 13min 41s, sys: 9.87 s, total: 13min 51s
Wall time: 15min 38s
18
'''

NameError: name 'get_filenames' is not defined

In [ ]:
quasardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/", extension='.fits')
batched_quasardata = np.array_split(np.array(quasardata),18)

%%time

noise = []
for i in range(len(batched_quasardata)):
    for j in range(len(batched_quasardata[0])):
        with fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/" +str(batched_quasardata[i][j])+ "") as hdul:
            data = hdul['SPALL'].data
            SN = data.field("SN_MEDIAN_ALL")
            noise.extend(SN)  #np.extend just adds another item to the list
            hdul.close()
            del hdul['SPALL'].data
            del hdul['COADD'].data
            del hdul['PRIMARY'].data   
#     hdul.close()

print(len(noise))

'''
7974
CPU times: user 14min 35s, sys: 12.5 s, total: 14min 47s
Wall time: 15min 29s
'''

In [3]:
quasardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/", extension='.fits')
batched_quasardata = np.array_split(np.array(quasardata),9)

%%time

noise = []
#flux = []
for i in range(len(batched_quasardata)):
    for j in range(len(batched_quasardata[0])):
        with fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/good_quasars/" +str(batched_quasardata[i][j])+ "", memmap = False) as hdul:
            data = hdul['SPALL'].data
            SN = data.field("SN_MEDIAN_ALL")
            noise.extend(SN)  #np.extend just adds another item to the list
            hdul.close()
            del hdul['SPALL'].data
            del hdul['COADD'].data
            del hdul['PRIMARY'].data   
#     hdul.close()

print(len(noise))

#this is the fastest for loop to obtain the noise data for the quasar spectra

NameError: name 'get_filenames' is not defined

In [ ]:
plt.hist(noise, bins= range(0,50), histtype = 'bar' )
plt.ylabel('Frequency')
plt.xlabel('Magnitude of S/N')
plt.title("Histogram of Noise of ~7000 Quasar Spectra")
plt.axvline(np.mean(noise), color='k', linestyle='dashed', linewidth=1, label = 'Mean')
plt.legend()
print('Noise mean = ', np.mean(noise))

# histograms of the noise data

In [ ]:
def get_filevalues(filename_list): 
    
    list_fluxarrays = []
    list_objtype = []
    list_noise = []
    list_wavelength = []
    list_redshift = []
    
    
    
    for i in range(len(stardata)):
        with fits.open("/Users/matt/Desktop/DESI_Research/DESI_ML/good_stars/" +str(stardata[i])+ "",\
                       memmap = False ) as hdul:
            
            data_c = hdul['COADD'].data
            data_s = hdul['SPALL'].data
            
            flux_val = data_c.field("flux")
            list_fluxarrays.append(flux_val) 
            
            objtype = data_s.field('OBJTYPE')
            list_objtype.append(objtype)
            
            noise_val = data_s.field('SN_MEDIAN_ALL')
            list_noise.append(noise_val)
            
            wavelength_val = data_c.field('loglam')
            list_wavelength.append(wavelength_val)
            
            redshift_val = data_s.field('Z')
            list_redshift.append(redshift_val)
            
            values = {'FLUX': list_fluxarrays, 'OBJTYPE': list_objtype, 'NOISE': list_noise,\
                      'WAVE': list_wavelength, 'REDSHIFT': list_redshift}
            
            hdul.close()
            del hdul['COADD'].data
            del hdul['PRIMARY'].data
            del hdul['SPALL'].data
            del hdul
            
    return values

In [ ]:
a,b,c = plt.hist(fluxlen_list, bins = 100)

In [ ]:
"""
Used in Quasar_data_inrange.ipynb
"""

# processes the flux data so it's averaged between 0 & 1
def preprocessing_data(values_dict): 
    
    maxlist = []
    filtered_flux_list = []
    for i in range(len(values_dict['FLUX'])):
        maxlist.append(max(values_dict['FLUX'][i]))
        
    maxval = max(maxlist)  # go back and fix this. The lists have to sum to 1
    
    for i in range(len(values_dict['FLUX'])):
        filtered_flux_list.append((values_dict['FLUX'][i])/maxval)
       
    return filtered_flux_list

# gives me the average length of all the flux arrays
def meanlength_flux_arrays(filtered_flux_list): 
    
    length = []
    for i in range(len(filtered_flux_arrays)):
        
        length.append(len(filtered_flux_arrays[i]))
        mean_length = np.mean(length)
        
    return mean_length 

In [ ]:
import pdb

pdb.pm()

# SUPER AMAZING. DEBUGGER!!! PYTHON DEBUGGER. pm stand for postmortem